In [1]:
import os
from typing import List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from sklearn import metrics
from torch import optim

# CNN

In [90]:
class Net(nn.Module):
    def __init__(self, in_features, dropout=0.25):
        super().__init__()

        def conv_out_len(layer, length_in):
            return (length_in + 2 * layer.padding[0] - layer.dilation[0] * (layer.kernel_size[0] - 1) - 1) // \
                   layer.stride[0] + 1

        self.conv1 = nn.Conv1d(in_features, 5, kernel_size=25, stride=1, padding=12)
        self.last = nn.Conv1d(self.conv1.out_channels, 1, kernel_size=1, stride=1, padding=0)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        # self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        x = x.squeeze(-1)
        x = self.relu(self.conv1(x))
        x = self.sigmoid(self.last(x))
        x = x.flatten(start_dim=1)
        return x

In [91]:
# Instantiate the model
net = Net(in_features=21)

In [92]:
pytorch_total_params = sum(p.numel() for p in net.parameters())
pytorch_total_params

2636

# RNN

In [81]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size=24, num_layers=2, bidirectional=True, dropout=0.2):
        super().__init__()

        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True,
                         bidirectional=bidirectional, nonlinearity='tanh', dropout=dropout)
        self.bidirectional = bidirectional
        
        if bidirectional:
            self.fc = nn.Linear(hidden_size*2, 1) # combine result from diff layers into one output for each AA
        else:
            self.fc = nn.Linear(hidden_size, 1)
            
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = x.transpose(1, 2)
        if self.bidirectional:
            h0 = torch.zeros(self.num_layers*2, x.size(0), self.hidden_size).to(device)
        else:
            h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
        # Forward Prop
        out, hn = self.rnn(x, h0)
        # all training example, last hidden state, all 
        # it is not the last hidden state, it is the last batch
        # print('out.squeeze() ', out.squeeze().size())
        out = self.fc(out)
        out = self.sigmoid(out)
        
        out = out.flatten(start_dim=1)
        return out

In [88]:
# Instantiate the model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net = Net(1024, num_layers=2, hidden_size=50, bidirectional=True).to(device)

In [89]:
pytorch_total_params = sum(p.numel() for p in net.parameters())
pytorch_total_params

122901